In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import warnings
from sklearn.model_selection import  train_test_split
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [15]:
data=pd.read_csv("E:/Exam/instagram_reach.csv")

In [16]:
data.head()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
0,0,1,mikequindazzi,Who are #DataScientist and what do they do? >>...,1600,#MachineLearning #AI #DataAnalytics #DataScien...,11 hours,139
1,1,2,drgorillapaints,We all know where it’s going. We just have to ...,880,#deck .#mac #macintosh#sayhello #apple #steve...,2 hours,23
2,2,3,aitrading_official,Alexander Barinov: 4 years as CFO in multinati...,255,#whoiswho #aitrading #ai #aitradingteam#instat...,2 hours,25
3,3,4,opensourcedworkplace,sfad,340,#iot #cre#workplace #CDO #bigdata #technology#...,3 hours,49
4,4,5,crea.vision,Ever missed a call while your phone was chargi...,304,#instamachinelearning #instabigdata#instamarke...,3 hours,30


In [17]:

data.drop(['S.No','Unnamed: 0','USERNAME','Caption','Hashtags'],axis=1,inplace=True)


In [18]:
data

,Followers,Time since posted,Likes
0,1600,11 hours,139
1,880,2 hours,23
2,255,2 hours,25
3,340,3 hours,49
4,304,3 hours,30
...,...,...,...
95,614,3 hours,31
96,450,3 hours,42
97,182,3 hours,10
98,2039,3 hours,222


In [21]:
data.isnull().sum()

Followers            0
Time since posted    0
Likes                0
dtype: int64

In [25]:
data['Time since posted'] = data['Time since posted'].map(lambda a: int(re.sub('hours', '', a)))

In [26]:
data

,Followers,Time since posted,Likes
0,1600,11,139
1,880,2,23
2,255,2,25
3,340,3,49
4,304,3,30
...,...,...,...
95,614,3,31
96,450,3,42
97,182,3,10
98,2039,3,222


In [28]:
X = data.drop(['Likes', 'Time since posted'], axis=1)
y = data[['Likes', 'Time since posted']]

In [30]:
X

,Followers
0,1600
1,880
2,255
3,340
4,304
...,...
95,614
96,450
97,182
98,2039


In [32]:
y

,Likes,Time since posted
0,139,11
1,23,2
2,25,2
3,49,3
4,30,3
...,...,...
95,31,3
96,42,3
97,10,3
98,222,3


In [45]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.multioutput import MultiOutputRegressor

In [46]:
models = {
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBRegressor": XGBRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
    }

In [49]:
def evaluate_clf(true, predicted):

    MAE = mean_absolute_error(true, predicted) # Calculate Mean Absolute Error
    MSE = mean_squared_error(true, predicted) # Calculate Mean Squared Error
    RMSE =np.sqrt(MSE) # Calculate Root Mean Squared Error
    r2score = r2_score(true, predicted)  # Calculate R_squaredScore/R2score
    return MAE, MSE, RMSE, r2score

In [50]:
def evaluate_models(X, y, models):
    
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)
    
    # Applying Satndard Normalization method (Z-Score Equation):
    scalar = StandardScaler()
    X_train = scalar.fit_transform(X_train)
    X_test = scalar.transform(X_test)
    
    models_list = []
    MAE_train_list = []
    MAE_test_list = []
    MSE_train_list = []
    MSE_test_list = []
    RMSE_train_list = []
    RMSE_test_list = []
    r2score_train_list = []
    r2score_test_list = []
    r2score_diff = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        wrapper = MultiOutputRegressor(model)
        wrapper.fit(X_train, y_train) # Train model

        # Make predictions
        y_train_pred = wrapper.predict(X_train)
        y_test_pred = wrapper.predict(X_test)

        # Training set performance
        model_train_MAE, model_train_MSE, model_train_RMSE,\
        model_train_r2score = evaluate_clf(y_train ,y_train_pred)
        
        MAE_train_list.append(model_train_MAE)
        MSE_train_list.append(model_train_MSE)
        RMSE_train_list.append(model_train_RMSE)
        r2score_train_list.append(model_train_r2score*100)
    
        # Test set performance
        model_test_MAE,model_test_MSE,model_test_RMSE,\
        model_test_r2score = evaluate_clf(y_test, y_test_pred)
        
        MAE_test_list.append(model_test_MAE)
        MSE_test_list.append(model_test_MSE)
        RMSE_test_list.append(model_test_RMSE)
        r2score_test_list.append(model_test_r2score*100)
        
        # r2score_train_test_difference
        r2score_diff.append(max(model_train_r2score, model_test_r2score) - 
                            min(model_train_r2score, model_test_r2score))  
    
        print(list(models.keys())[i])
        models_list.append(list(models.keys())[i])

        print('Model performance for Training set')
        print("- MAE: {:.4f}".format(model_train_MAE))
        print('- MSE: {:.4f}'.format(model_train_MSE)) 
        print('- RMSE: {:.4f}'.format(model_train_RMSE))
        print('- r2score: {:.4f}'.format(model_train_r2score))

        print('----------------------------------')

        print('Model performance for Test set')
        print('- MAE: {:.4f}'.format(model_test_MAE))
        print('- MSE: {:.4f}'.format(model_test_MSE))
        print('- RMSE: {:.4f}'.format(model_test_RMSE))
        print('- r2score: {:.4f}'.format(model_test_r2score))
        
        print('='*35)
        print('\n')    
        
    report = pd.DataFrame(list(zip(models_list, MAE_train_list, MAE_test_list, MSE_train_list, MSE_test_list, RMSE_train_list, RMSE_test_list, r2score_train_list,  r2score_test_list, r2score_diff)), columns=['Model Name', 'MAE_train', 'MAE_test', 'MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'r2score_train', 'r2score_test','r2score_diff']).sort_values(by='r2score_test',ascending=False)
        
    return report

In [51]:
report = evaluate_models(X, y, models)

AdaBoost Regressor
Model performance for Training set
- MAE: 14.4889
- MSE: 622.9411
- RMSE: 24.9588
- r2score: 0.7318
----------------------------------
Model performance for Test set
- MAE: 12.7299
- MSE: 487.4499
- RMSE: 22.0783
- r2score: 0.2489


Gradient Boosting
Model performance for Training set
- MAE: 4.4906
- MSE: 56.4654
- RMSE: 7.5143
- r2score: 0.9697
----------------------------------
Model performance for Test set
- MAE: 5.7064
- MSE: 95.7683
- RMSE: 9.7861
- r2score: 0.5670


XGBRegressor
Model performance for Training set
- MAE: 0.5929
- MSE: 2.6355
- RMSE: 1.6234
- r2score: 0.9931
----------------------------------
Model performance for Test set
- MAE: 12.0721
- MSE: 772.2032
- RMSE: 27.7885
- r2score: -0.0427


Decision Tree Regressor
Model performance for Training set
- MAE: 0.3438
- MSE: 2.4469
- RMSE: 1.5642
- r2score: 0.9932
----------------------------------
Model performance for Test set
- MAE: 5.9000
- MSE: 97.5500
- RMSE: 9.8767
- r2score: 0.4991


Random For

In [52]:
final_model = RandomForestRegressor(n_estimators = 1000, max_depth = 5, random_state=42, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)
wrapper = MultiOutputRegressor(final_model)

final_model = wrapper.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

print("Final RandomForest Regressor Accuracy Score (Train) :", final_model.score(X_train,y_train))
print("Final RandomForest Regressor Accuracy Score (Test) :", r2_score(y_pred,y_test))

Final RandomForest Regressor Accuracy Score (Train) : 0.72373720446705
Final RandomForest Regressor Accuracy Score (Test) : 0.13164184324864314


In [53]:
# Let's predict:
print(X_test[2])
y_pred = final_model.predict([X_test[4]])
y_pred
print("Nnumber of Likes: ", y_pred[0][0])
print("Time since posted: ", y_pred[0][1], 'hours')

[0.15081552]
Nnumber of Likes:  18.227091901403544
Time since posted:  2.0846796826018674 hours
